In [1]:
from sklearn.preprocessing import StandardScaler

In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [4]:
holidays = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/holidays.csv")
sub = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/sample_submission.csv")
test = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/test.csv")
train = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/train.csv")
weather = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/weather.csv")

In [5]:
weather

,date,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,name
0,2021-01-01 00:00:00,38.6184,28.6712,7.6,10.5,0.0,86.1,173.3,2.6,1.0,5.9,Manisa-Ahmetli
1,2021-01-01 01:00:00,38.6184,28.6712,7.4,6.8,0.0,88.0,174.5,2.7,1.0,5.7,Manisa-Ahmetli
2,2021-01-01 02:00:00,38.6184,28.6712,7.3,4.3,0.0,90.0,177.5,2.8,1.0,5.4,Manisa-Ahmetli
3,2021-01-01 03:00:00,38.6184,28.6712,7.2,8.9,0.0,90.8,175.3,2.7,1.0,5.3,Manisa-Ahmetli
4,2021-01-01 04:00:00,38.6184,28.6712,8.0,22.1,0.0,89.7,172.1,2.5,1.0,6.5,Manisa-Ahmetli
...,...,...,...,...,...,...,...,...,...,...,...,...
1302835,2024-02-29 19:00:00,38.2283,27.9748,16.8,50.0,0.0,61.6,278.2,1.7,16.0,16.3,Izmir-Odemis
1302836,2024-02-29 20:00:00,38.2283,27.9748,15.4,52.4,0.0,69.4,264.7,1.4,9.2,15.4,Izmir-Odemis
1302837,2024-02-29 21:00:00,38.2283,27.9748,16.5,63.7,0.0,62.9,278.4,0.5,43.9,16.9,Izmir-Odemis
1302838,2024-02-29 22:00:00,38.2283,27.9748,15.9,61.7,0.0,64.3,242.1,0.7,58.5,16.1,Izmir-Odemis


### Yeni veri setinde mart ayındaki 10 günlük veri silinmiş.. Yani mart ayından bilgi yok. Farklı bir veri seti kullanacağız diyorlar ama bence öyle olmayacak

In [6]:
# Tarih sütununu datetime tipine dönüştürelim
weather['date'] = pd.to_datetime(weather['date'])

In [7]:
weather['name'] = weather['name'].str.lower()

In [8]:
weather.rename(columns={'name': 'ilce'}, inplace=True)

# Weather'deki global_rad:W sütunu, bize gün ışığının olduğu saatleri verecek.

In [9]:
# 'global_rad:W' sütununa göre gündüz ve gece gruplamalarını yapalım
weather['gece'] = weather['global_rad:W'] == 0
weather['gündüz'] = weather['global_rad:W'] != 0

# Tarih ve ilçe sütunlarına göre gruplayıp, her bir grup için gündüz ve gece verilerinin ortalamasını alalım
grouped = weather.groupby(['date', 'ilce', 'gece', 'gündüz'])
gece_ortalama = grouped.mean().loc[(slice(None), slice(None), True, False), :].reset_index(level=[2, 3], drop=True).add_prefix('gece_')
gündüz_ortalama = grouped.mean().loc[(slice(None), slice(None), False, True), :].reset_index(level=[2, 3], drop=True).add_prefix('gündüz_')

# Her bir gün için sadeleştirilmiş veriyi elde edelim
sadelesmis_df = pd.concat([gece_ortalama, gündüz_ortalama], axis=1).reset_index()

In [10]:
sadelesmis_df = sadelesmis_df.groupby([pd.Grouper(key='date', freq='D'), 'ilce']).mean().reset_index()

In [11]:
len(sadelesmis_df)

54285

In [12]:
sadelesmis_df[sadelesmis_df['ilce'] == "manisa-akhisar"].sort_values(by='date').head(1)

,date,ilce,gece_lat,gece_lon,gece_t_2m:C,gece_effective_cloud_cover:p,gece_global_rad:W,gece_relative_humidity_2m:p,gece_wind_dir_10m:d,gece_wind_speed_10m:ms,...,gündüz_lat,gündüz_lon,gündüz_t_2m:C,gündüz_effective_cloud_cover:p,gündüz_global_rad:W,gündüz_relative_humidity_2m:p,gündüz_wind_dir_10m:d,gündüz_wind_speed_10m:ms,gündüz_prob_precip_1h:p,gündüz_t_apparent:C
31,2021-01-01,manisa-akhisar,38.9177,27.8366,9.886667,51.96,0.0,88.493333,104.1,1.413333,...,38.9177,27.8366,13.211111,55.377778,190.822222,82.866667,182.711111,1.2,1.0,15.5


In [13]:
sadelesmis_df.isna().sum()

date                              0
ilce                              0
gece_lat                          0
gece_lon                          0
gece_t_2m:C                       0
gece_effective_cloud_cover:p      0
gece_global_rad:W                 0
gece_relative_humidity_2m:p       0
gece_wind_dir_10m:d               0
gece_wind_speed_10m:ms            0
gece_prob_precip_1h:p             0
gece_t_apparent:C                 0
gündüz_lat                        0
gündüz_lon                        0
gündüz_t_2m:C                     0
gündüz_effective_cloud_cover:p    0
gündüz_global_rad:W               0
gündüz_relative_humidity_2m:p     0
gündüz_wind_dir_10m:d             0
gündüz_wind_speed_10m:ms          0
gündüz_prob_precip_1h:p           0
gündüz_t_apparent:C               0
dtype: int64

# Tüm günleri ekleyelim. haftaiçi veya sonu olarak da ekleyelim.

In [14]:
# Hafta içi ve hafta sonu sütunlarını ekleyelim
sadelesmis_df['hafta_ici'] = (sadelesmis_df['date'].dt.dayofweek < 5).astype(int)
sadelesmis_df['hafta_sonu'] = (sadelesmis_df['date'].dt.dayofweek >= 5).astype(int)

In [15]:
günler = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for gün in günler:
    sadelesmis_df[gün] = (sadelesmis_df['date'].dt.strftime('%A') == gün).astype(int)

In [16]:
# 'ilce' ve 'tarih' sütunlarını oluşturarak eksik satırları doldurmak için kullanacağımız bir DataFrame oluşturalım
ilceler = train['ilce'].unique()
tarihler = pd.date_range(start='2021-01-01', end='2024-02-29', freq='D')
takvim = pd.DataFrame([(ilce, tarih) for ilce in ilceler for tarih in tarihler], columns=['ilce', 'tarih'])

# 'takvim' DataFrame'ini 'train' DataFrame'ine ekleyelim ve eksik olan satırları dolduralım
train['tarih'] = pd.to_datetime(train['tarih'])  # 'train' DataFrame'indeki 'tarih' sütununu datetime64[ns] veri türüne dönüştürme
takvim['tarih'] = pd.to_datetime(takvim['tarih'])  # 'takvim' DataFrame'indeki 'tarih' sütununu datetime64[ns] veri türüne dönüştürme
train = pd.merge(takvim, train, on=['ilce', 'tarih'], how='left')

# Eksik olan satırları dolduralım
train['bildirimsiz_sum'].fillna(0, inplace=True)
train['bildirimli_sum'].fillna(0, inplace=True)

/tmp/ipykernel_18/947199136.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['bildirimsiz_sum'].fillna(0, inplace=True)
/tmp/ipykernel_18/947199136.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [17]:
sadelesmis_df = sadelesmis_df.rename(columns={'date': 'tarih'})

In [18]:
train['tarih'] = pd.to_datetime(train['tarih'])
sadelesmis_df['tarih'] = pd.to_datetime(sadelesmis_df['tarih'])

In [19]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [20]:
merged_df = pd.merge(train, sadelesmis_df, on=['tarih','ilce'], how='left')

In [21]:
# Her bir unique ilçe için işlem yapma
for ilce in merged_df['ilce'].unique():
    ilce_verileri = merged_df[merged_df['ilce'] == ilce].copy()  # ilgili ilçeye ait verileri seçme ve kopyalama
    ilce_verileri['degisim'] = ilce_verileri['bildirimsiz_sum'].diff()  # Bildirimsiz sum sütununda değişimi hesaplama
    ilce_verileri['oran'] = ilce_verileri['bildirimsiz_sum'].pct_change() * 100  # Değişim oranını yüzde olarak hesaplama
    ilce_verileri['degisim_ilk_deger'] = ilce_verileri['bildirimsiz_sum'] - ilce_verileri['bildirimsiz_sum'].iloc[0]  # İlk değere göre değişimi hesaplama
    merged_df.loc[merged_df['ilce'] == ilce, 'degisim'] = ilce_verileri['degisim']
    merged_df.loc[merged_df['ilce'] == ilce, 'oran'] = ilce_verileri['oran']
    merged_df.loc[merged_df['ilce'] == ilce, 'degisim_ilk_deger'] = ilce_verileri['degisim_ilk_deger']

merged_df.fillna(0, inplace=True)

In [22]:
merged_df['oran'].replace([float('inf'), -float('inf')], 0, inplace=True)
merged_df['oran'] = merged_df['oran'].astype(float).fillna(0).astype(int)

/tmp/ipykernel_18/1343230000.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['oran'].replace([float('inf'), -float('inf')], 0, inplace=True)


In [23]:
# Sütun adlarını değiştirme
holidays.rename(columns={'Yıl': 'year', 'Ay': 'month', 'Gün': 'day'}, inplace=True)

# Yıl, Ay ve Gün sütunlarını birleştirme
holidays['tarih'] = pd.to_datetime(holidays[['year', 'month', 'day']])

# Tarih sütunlarını birleştirmeye artık gerek yok, bu yüzden sütunları temizleme
holidays.drop(['year', 'month', 'day'], axis=1, inplace=True)

In [24]:
merged_df['tarih'] = pd.to_datetime(merged_df['tarih'])
holidays['tarih'] = pd.to_datetime(holidays['tarih'])

# holidays veri çerçevesindeki tarihlerin merged_df içinde olup olmadığını kontrol etme
merged_df['ozel_gun'] = merged_df['tarih'].isin(holidays['tarih']).astype(int)

In [25]:
izmir_ilceleri = merged_df[merged_df['ilce'].str.startswith('izmir')]
manisa_ilceleri = merged_df[merged_df['ilce'].str.startswith('manisa')]

izmir_gunluk = izmir_ilceleri.groupby('tarih').agg({'bildirimli_sum': 'sum', 'bildirimsiz_sum': 'sum'}).reset_index()
manisa_gunluk = manisa_ilceleri.groupby('tarih').agg({'bildirimli_sum': 'sum', 'bildirimsiz_sum': 'sum'}).reset_index()

In [26]:
# İzmir ve Manisa ilçelerindeki günlük toplam bildirimli ve bildirimsiz değerleri merged_df'e birleştirme
merged_df['izmir_toplam_bildirimli'] = merged_df['tarih'].map(izmir_gunluk.set_index('tarih')['bildirimli_sum'])
merged_df['izmir_toplam_bildirimsiz'] = merged_df['tarih'].map(izmir_gunluk.set_index('tarih')['bildirimsiz_sum'])
merged_df['manisa_toplam_bildirimli'] = merged_df['tarih'].map(manisa_gunluk.set_index('tarih')['bildirimli_sum'])
merged_df['manisa_toplam_bildirimsiz'] = merged_df['tarih'].map(manisa_gunluk.set_index('tarih')['bildirimsiz_sum'])

In [27]:
# BUNU YAPMADAN EVVEL AYIRIYORUZ...


# İstenen tarih aralığı
start_date = '2023-01-01'
end_date = '2024-01-03'

test_start_date = '2024-01-04'
test_end_date = '2024-01-31'

# Tarih aralığına göre veri setini filtreleme
test_merged = merged_df[(merged_df['tarih'] >= test_start_date) & (merged_df['tarih'] <= test_end_date)]
merged_df = merged_df[(merged_df['tarih'] >= start_date) & (merged_df['tarih'] <= end_date)]

In [28]:
"""from sklearn.preprocessing import StandardScaler

# Veri setinizden X değerlerini seçme
X = merged_df.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])  # 'tarih', 'ilce' ve 'bildirimsiz_sum' dışındaki sütunlar X değerleridir

# Standardizasyon işlemi için StandardScaler nesnesi oluşturma
scaler = StandardScaler()

# X değerlerini standardize etme
X_standardized = scaler.fit_transform(X)

# Sonuçları kullanma
# X_standardized artık standartize edilmiş X değerlerini içeren bir NumPy dizisidir
buna gerek yok...
"""

"from sklearn.preprocessing import StandardScaler\n\n# Veri setinizden X değerlerini seçme\nX = merged_df.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])  # 'tarih', 'ilce' ve 'bildirimsiz_sum' dışındaki sütunlar X değerleridir\n\n# Standardizasyon işlemi için StandardScaler nesnesi oluşturma\nscaler = StandardScaler()\n\n# X değerlerini standardize etme\nX_standardized = scaler.fit_transform(X)\n\n# Sonuçları kullanma\n# X_standardized artık standartize edilmiş X değerlerini içeren bir NumPy dizisidir\nbuna gerek yok...\n"

# Ortalamanın etrafına sıkıştırma kodu:

In [29]:
"""# Aykırı değerleri belirleme ve üst eşiğe eşitleme
threshold_factor = 0.1  # Standart sapmanın kaç katı olacak
outliers_info = {}

for ilce in merged_df['ilce'].unique():
    ilce_data = merged_df[merged_df['ilce'] == ilce]
    ilce_mean = ilce_data['bildirimsiz_sum'].mean()
    ilce_std = ilce_data['bildirimsiz_sum'].std()
    outliers_upper = ilce_data[ilce_data['bildirimsiz_sum'] > ilce_mean + threshold_factor * ilce_std]
    outliers_lower = ilce_data[ilce_data['bildirimsiz_sum'] < ilce_mean - threshold_factor * ilce_std]
    outliers_count_upper = len(outliers_upper)
    outliers_count_lower = len(outliers_lower)
    outliers_info[ilce] = (outliers_count_upper, outliers_count_lower)

    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme
    merged_df.loc[outliers_upper.index, 'bildirimsiz_sum'] = ilce_mean
    merged_df.loc[outliers_lower.index, 'bildirimsiz_sum'] = ilce_mean

# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma
print("İlçe Aykırı Değer Bilgileri:")
for ilce, counts in outliers_info.items():
    print(f"{ilce}: {counts[0]} adet üst, {counts[1]} adet alt aykırı değer tespit edildi.")
"""

'# Aykırı değerleri belirleme ve üst eşiğe eşitleme\nthreshold_factor = 0.1  # Standart sapmanın kaç katı olacak\noutliers_info = {}\n\nfor ilce in merged_df[\'ilce\'].unique():\n    ilce_data = merged_df[merged_df[\'ilce\'] == ilce]\n    ilce_mean = ilce_data[\'bildirimsiz_sum\'].mean()\n    ilce_std = ilce_data[\'bildirimsiz_sum\'].std()\n    outliers_upper = ilce_data[ilce_data[\'bildirimsiz_sum\'] > ilce_mean + threshold_factor * ilce_std]\n    outliers_lower = ilce_data[ilce_data[\'bildirimsiz_sum\'] < ilce_mean - threshold_factor * ilce_std]\n    outliers_count_upper = len(outliers_upper)\n    outliers_count_lower = len(outliers_lower)\n    outliers_info[ilce] = (outliers_count_upper, outliers_count_lower)\n\n    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme\n    merged_df.loc[outliers_upper.index, \'bildirimsiz_sum\'] = ilce_mean\n    merged_df.loc[outliers_lower.index, \'bildirimsiz_sum\'] = ilce_mean\n\n# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma\np

In [30]:
"""# Aykırı değerleri belirleme ve üst eşiğe eşitleme
threshold_factor = 0.1  # Standart sapmanın kaç katı olacak
outliers_info = {}

for ilce in merged_df['ilce'].unique():
    ilce_data = merged_df[merged_df['ilce'] == ilce]
    ilce_mean = ilce_data['bildirimsiz_sum'].mean()
    outliers = ilce_data[ilce_data['bildirimsiz_sum'] > ilce_mean]
    outliers_count = len(outliers)
    outliers_info[ilce] = outliers_count

    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme
    merged_df.loc[outliers.index, 'bildirimsiz_sum'] = ilce_mean

# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma
print("İlçe Aykırı Değer Bilgileri:")
for ilce, count in outliers_info.items():
    print(f"{ilce}: {count} adet aykırı değer tespit edildi.")"""

'# Aykırı değerleri belirleme ve üst eşiğe eşitleme\nthreshold_factor = 0.1  # Standart sapmanın kaç katı olacak\noutliers_info = {}\n\nfor ilce in merged_df[\'ilce\'].unique():\n    ilce_data = merged_df[merged_df[\'ilce\'] == ilce]\n    ilce_mean = ilce_data[\'bildirimsiz_sum\'].mean()\n    outliers = ilce_data[ilce_data[\'bildirimsiz_sum\'] > ilce_mean]\n    outliers_count = len(outliers)\n    outliers_info[ilce] = outliers_count\n\n    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme\n    merged_df.loc[outliers.index, \'bildirimsiz_sum\'] = ilce_mean\n\n# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma\nprint("İlçe Aykırı Değer Bilgileri:")\nfor ilce, count in outliers_info.items():\n    print(f"{ilce}: {count} adet aykırı değer tespit edildi.")'

# MODEL = Elastic.net

In [31]:
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

# Tahmin edilecek tarih aralığı
start_date = '2024-01-04'
end_date = '2024-01-31'

# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma
train_data = merged_df

# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma
test_data = test_merged

# Tahminde bulunacak ilçelerin listesi
ilceler = merged_df['ilce'].unique()

# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım
predictions = {}

# StandardScaler nesnesi oluşturma ve eğitme
scaler = StandardScaler()
scaler.fit(train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))

# Her bir ilçe için tahminde bulunma
for ilce in ilceler:
    ilce_train_data = train_data[train_data['ilce'] == ilce]
    ilce_test_data = test_data[test_data['ilce'] == ilce]
    
    # Standartlaştırma işlemi
    features_train_standardized = scaler.transform(ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    features_test_standardized = scaler.transform(ilce_test_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    
    # Hedef değişken
    target = ilce_train_data['bildirimsiz_sum']
    
    # Modeli eğitme (ElasticNet regresyonu)
    model = ElasticNet(alpha=0.1, l1_ratio=0.5)  # İstediğiniz alpha ve l1_ratio değerlerini belirtin
    model.fit(features_train_standardized, target)
    
    # Tahminde bulunma
    ilce_predictions = model.predict(features_test_standardized)
    
    # Tahminleri depolama
    predictions[ilce] = ilce_predictions


In [32]:
"""from sklearn.linear_model import ElasticNet

# Tahmin edilecek tarih aralığı
start_date = '2024-01-04'
end_date = '2024-01-31'

# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma
train_data = merged_df

# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma
test_data = test_merged

# Tahminde bulunacak ilçelerin listesi
ilceler = merged_df['ilce'].unique()

# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım
predictions = {}

# Her bir ilçe için tahminde bulunma
for ilce in ilceler:
    ilce_train_data = train_data[train_data['ilce'] == ilce]
    ilce_test_data = test_data[test_data['ilce'] == ilce]
    
    # Tahminde kullanılacak özellikler (tüm sütunlar)
    features = ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])
    
    # Hedef değişken
    target = ilce_train_data['bildirimsiz_sum']
    
    # Modeli eğitme (ElasticNet regresyonu)
    model = ElasticNet(alpha=0.1, l1_ratio=0.1)  # İstediğiniz alpha ve l1_ratio değerlerini belirtin
    model.fit(features, target)
    
    # Tahminde bulunma
    ilce_predictions = model.predict(ilce_test_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    
    # Tahminleri depolama
    predictions[ilce] = ilce_predictions"""

"from sklearn.linear_model import ElasticNet\n\n# Tahmin edilecek tarih aralığı\nstart_date = '2024-01-04'\nend_date = '2024-01-31'\n\n# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma\ntrain_data = merged_df\n\n# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma\ntest_data = test_merged\n\n# Tahminde bulunacak ilçelerin listesi\nilceler = merged_df['ilce'].unique()\n\n# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım\npredictions = {}\n\n# Her bir ilçe için tahminde bulunma\nfor ilce in ilceler:\n    ilce_train_data = train_data[train_data['ilce'] == ilce]\n    ilce_test_data = test_data[test_data['ilce'] == ilce]\n    \n    # Tahminde kullanılacak özellikler (tüm sütunlar)\n    features = ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])\n    \n    # Hedef değişken\n    target = ilce_train_data['bildirimsiz_sum']\n    \n    # Modeli eğitme (ElasticNet regresyonu)\n    model = ElasticNet(alpha=0.1, l

In [33]:
from sklearn.metrics import mean_absolute_error

# MAE skorlarını depolamak için bir sözlük oluşturalım
mae_scores = {}

# Her bir ilçe için MAE skorunu hesaplayalım
for ilce, ilce_predictions in predictions.items():
    true_values = test_data[test_data['ilce'] == ilce]['bildirimsiz_sum'].values
    mae = mean_absolute_error(true_values, ilce_predictions)
    mae_scores[ilce] = mae

# Her ilçenin MAE skorunu yazdıralım
print("İlçe Bazlı MAE Skorları:")
for ilce, mae in mae_scores.items():
    print(f"{ilce}: {mae}")

İlçe Bazlı MAE Skorları:
izmir-aliaga: 0.7342781573154549
izmir-bayindir: 0.33413954367671156
izmir-bayrakli: 0.792450373446296
izmir-bergama: 0.3875733738713784
izmir-bornova: 0.2844806274757237
izmir-buca: 0.3503828304113324
izmir-cesme: 0.16299576738576124
izmir-cigli: 0.33292005850463546
izmir-dikili: 0.29562765597512347
izmir-foca: 0.3973074299544327
izmir-gaziemir: 0.45590263567290884
izmir-guzelbahce: 0.6630670502264356
izmir-karabaglar: 0.5855921612869979
izmir-karaburun: 0.9338144322023008
izmir-karsiyaka: 0.35049710169738674
izmir-kemalpasa: 0.4843304903117676
izmir-kinik: 0.6954962437073245
izmir-kiraz: 0.444688928671835
izmir-konak: 0.17949558538666613
izmir-menderes: 0.43918424650136645
izmir-menemen: 0.5211879652512625
izmir-odemis: 0.294748879079035
izmir-seferihisar: 0.6128428414732695
izmir-tire: 0.2413552072487804
izmir-torbali: 0.35626637878636735
izmir-urla: 0.3146666601399963
manisa-akhisar: 0.2957050958244629
manisa-alasehir: 0.35655267996274087
manisa-demirci: 0.

In [34]:
# Tüm MAE skorlarının ortalamasını hesaplayalım
overall_mae = sum(mae_scores.values()) / len(mae_scores)

# Ortalama MAE skorunu yazdıralım
print("Tüm İlçelerin Ortalama MAE Skoru:", overall_mae)

Tüm İlçelerin Ortalama MAE Skoru: 0.45437154197519336


In [35]:
print("0.01-0.01 ikisi: ", 0.6536862632354179)

0.01-0.01 ikisi:  0.6536862632354179


#### rmse

In [36]:
from sklearn.metrics import mean_squared_error

ilce_rmse = {}
for ilce, ilce_test_data in test_data.groupby('ilce'):
    true_values = ilce_test_data['bildirimsiz_sum']
    
    predicted_values = predictions[ilce]
    
    rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    
    ilce_rmse[ilce] = rmse

for ilce, rmse in ilce_rmse.items():
    print(f"{ilce}: RMSE = {rmse}")

izmir-aliaga: RMSE = 0.9383779597440948
izmir-balcova: RMSE = 0.5849070122875636
izmir-bayindir: RMSE = 0.4715190784075809
izmir-bayrakli: RMSE = 1.1687650897556545
izmir-bergama: RMSE = 0.5765937821329224
izmir-beydag: RMSE = 0.940423802724218
izmir-bornova: RMSE = 0.3839459729904814
izmir-buca: RMSE = 0.42665716055205194
izmir-cesme: RMSE = 0.20860624892680224
izmir-cigli: RMSE = 0.4214619157232861
izmir-dikili: RMSE = 0.4367672138047879
izmir-foca: RMSE = 0.516040440936699
izmir-gaziemir: RMSE = 0.5839444016223643
izmir-guzelbahce: RMSE = 0.877567551207319
izmir-karabaglar: RMSE = 0.7128261779661937
izmir-karaburun: RMSE = 1.114049221780601
izmir-karsiyaka: RMSE = 0.4546665387762321
izmir-kemalpasa: RMSE = 0.6079138157407135
izmir-kinik: RMSE = 1.0545264263552772
izmir-kiraz: RMSE = 0.6924919539253819
izmir-konak: RMSE = 0.21929325579360007
izmir-menderes: RMSE = 0.501215403159864
izmir-menemen: RMSE = 0.7893352811835207
izmir-narlidere: RMSE = 0.8970278277461721
izmir-odemis: RMSE 

In [37]:
# Tüm ilçelerin RMSE değerlerinin ortalamasını hesapla
mean_rmse = np.mean(list(ilce_rmse.values()))

# Ortalama RMSE değerini yazdır
print("Tüm ilçelerin Ortalama RMSE değeri:", mean_rmse)

Tüm ilçelerin Ortalama RMSE değeri: 0.6015945681580915


In [38]:
print("0.01-0.01 ikisi: ", 0.865188921845218)

0.01-0.01 ikisi:  0.865188921845218


# Submission

In [39]:
holidays = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/holidays.csv")
sub = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/sample_submission.csv")
test = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/test.csv")
train = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/train.csv")
weather = pd.read_csv("/kaggle/input/gdz-elektrik-datathon-2024/weather.csv")

In [40]:
weather['date'] = pd.to_datetime(weather['date'])

In [41]:
weather['name'] = weather['name'].str.lower()

In [42]:
weather.rename(columns={'name': 'ilce'}, inplace=True)

In [43]:
# 'global_rad:W' sütununa göre gündüz ve gece gruplamalarını yapalım
weather['gece'] = weather['global_rad:W'] == 0
weather['gündüz'] = weather['global_rad:W'] != 0

# Tarih ve ilçe sütunlarına göre gruplayıp, her bir grup için gündüz ve gece verilerinin ortalamasını alalım
grouped = weather.groupby(['date', 'ilce', 'gece', 'gündüz'])
gece_ortalama = grouped.mean().loc[(slice(None), slice(None), True, False), :].reset_index(level=[2, 3], drop=True).add_prefix('gece_')
gündüz_ortalama = grouped.mean().loc[(slice(None), slice(None), False, True), :].reset_index(level=[2, 3], drop=True).add_prefix('gündüz_')

# Her bir gün için sadeleştirilmiş veriyi elde edelim
sadelesmis_df = pd.concat([gece_ortalama, gündüz_ortalama], axis=1).reset_index()

In [44]:
sadelesmis_df = sadelesmis_df.groupby([pd.Grouper(key='date', freq='D'), 'ilce']).mean().reset_index()

In [45]:
sadelesmis_df['hafta_ici'] = (sadelesmis_df['date'].dt.dayofweek < 5).astype(int)
sadelesmis_df['hafta_sonu'] = (sadelesmis_df['date'].dt.dayofweek >= 5).astype(int)

In [46]:
günler = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for gün in günler:
    sadelesmis_df[gün] = (sadelesmis_df['date'].dt.strftime('%A') == gün).astype(int)

In [47]:
ilceler = train['ilce'].unique()
tarihler = pd.date_range(start='2021-01-01', end='2024-02-29', freq='D')
takvim = pd.DataFrame([(ilce, tarih) for ilce in ilceler for tarih in tarihler], columns=['ilce', 'tarih'])

# 'takvim' DataFrame'ini 'train' DataFrame'ine ekleyelim ve eksik olan satırları dolduralım
train['tarih'] = pd.to_datetime(train['tarih'])  # 'train' DataFrame'indeki 'tarih' sütununu datetime64[ns] veri türüne dönüştürme
takvim['tarih'] = pd.to_datetime(takvim['tarih'])  # 'takvim' DataFrame'indeki 'tarih' sütununu datetime64[ns] veri türüne dönüştürme
train = pd.merge(takvim, train, on=['ilce', 'tarih'], how='left')

# Eksik olan satırları dolduralım
train['bildirimsiz_sum'].fillna(0, inplace=True)
train['bildirimli_sum'].fillna(0, inplace=True)

/tmp/ipykernel_18/2317103827.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['bildirimsiz_sum'].fillna(0, inplace=True)
/tmp/ipykernel_18/2317103827.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

In [48]:
sadelesmis_df = sadelesmis_df.rename(columns={'date': 'tarih'})

In [49]:
train['tarih'] = pd.to_datetime(train['tarih'])
sadelesmis_df['tarih'] = pd.to_datetime(sadelesmis_df['tarih'])

In [50]:
merged_df = pd.merge(train, sadelesmis_df, on=['tarih','ilce'], how='left')

In [51]:
# Her bir unique ilçe için işlem yapma
for ilce in merged_df['ilce'].unique():
    ilce_verileri = merged_df[merged_df['ilce'] == ilce].copy()  # ilgili ilçeye ait verileri seçme ve kopyalama
    ilce_verileri['degisim'] = ilce_verileri['bildirimsiz_sum'].diff()  # Bildirimsiz sum sütununda değişimi hesaplama
    ilce_verileri['oran'] = ilce_verileri['bildirimsiz_sum'].pct_change() * 100  # Değişim oranını yüzde olarak hesaplama
    ilce_verileri['degisim_ilk_deger'] = ilce_verileri['bildirimsiz_sum'] - ilce_verileri['bildirimsiz_sum'].iloc[0]  # İlk değere göre değişimi hesaplama
    merged_df.loc[merged_df['ilce'] == ilce, 'degisim'] = ilce_verileri['degisim']
    merged_df.loc[merged_df['ilce'] == ilce, 'oran'] = ilce_verileri['oran']
    merged_df.loc[merged_df['ilce'] == ilce, 'degisim_ilk_deger'] = ilce_verileri['degisim_ilk_deger']

merged_df.fillna(0, inplace=True)

In [52]:
merged_df['oran'].replace([float('inf'), -float('inf')], 0, inplace=True)
merged_df['oran'] = merged_df['oran'].astype(float).fillna(0).astype(int)

/tmp/ipykernel_18/1343230000.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df['oran'].replace([float('inf'), -float('inf')], 0, inplace=True)


In [53]:
# Sütun adlarını değiştirme
holidays.rename(columns={'Yıl': 'year', 'Ay': 'month', 'Gün': 'day'}, inplace=True)

# Yıl, Ay ve Gün sütunlarını birleştirme
holidays['tarih'] = pd.to_datetime(holidays[['year', 'month', 'day']])

# Tarih sütunlarını birleştirmeye artık gerek yok, bu yüzden sütunları temizleme
holidays.drop(['year', 'month', 'day'], axis=1, inplace=True)

In [54]:
merged_df['tarih'] = pd.to_datetime(merged_df['tarih'])
holidays['tarih'] = pd.to_datetime(holidays['tarih'])

# holidays veri çerçevesindeki tarihlerin merged_df içinde olup olmadığını kontrol etme
merged_df['ozel_gun'] = merged_df['tarih'].isin(holidays['tarih']).astype(int)

In [55]:
izmir_ilceleri = merged_df[merged_df['ilce'].str.startswith('izmir')]
manisa_ilceleri = merged_df[merged_df['ilce'].str.startswith('manisa')]

izmir_gunluk = izmir_ilceleri.groupby('tarih').agg({'bildirimli_sum': 'sum', 'bildirimsiz_sum': 'sum'}).reset_index()
manisa_gunluk = manisa_ilceleri.groupby('tarih').agg({'bildirimli_sum': 'sum', 'bildirimsiz_sum': 'sum'}).reset_index()

In [56]:
# İzmir ve Manisa ilçelerindeki günlük toplam bildirimli ve bildirimsiz değerleri merged_df'e birleştirme
merged_df['izmir_toplam_bildirimli'] = merged_df['tarih'].map(izmir_gunluk.set_index('tarih')['bildirimli_sum'])
merged_df['izmir_toplam_bildirimsiz'] = merged_df['tarih'].map(izmir_gunluk.set_index('tarih')['bildirimsiz_sum'])
merged_df['manisa_toplam_bildirimli'] = merged_df['tarih'].map(manisa_gunluk.set_index('tarih')['bildirimli_sum'])
merged_df['manisa_toplam_bildirimsiz'] = merged_df['tarih'].map(manisa_gunluk.set_index('tarih')['bildirimsiz_sum'])

In [57]:
"""# Aykırı değerleri belirleme ve üst eşiğe eşitleme
threshold_factor = 0.1  # Standart sapmanın kaç katı olacak
outliers_info = {}

for ilce in sub_train['ilce'].unique():
    ilce_data = sub_train[sub_train['ilce'] == ilce]
    ilce_mean = ilce_data['bildirimsiz_sum'].mean()
    outliers = ilce_data[ilce_data['bildirimsiz_sum'] > ilce_mean]
    outliers_count = len(outliers)
    outliers_info[ilce] = outliers_count

    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme
    sub_train.loc[outliers.index, 'bildirimsiz_sum'] = ilce_mean

# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma
print("İlçe Aykırı Değer Bilgileri:")
for ilce, count in outliers_info.items():
    print(f"{ilce}: {count} adet aykırı değer tespit edildi.")"""

'# Aykırı değerleri belirleme ve üst eşiğe eşitleme\nthreshold_factor = 0.1  # Standart sapmanın kaç katı olacak\noutliers_info = {}\n\nfor ilce in sub_train[\'ilce\'].unique():\n    ilce_data = sub_train[sub_train[\'ilce\'] == ilce]\n    ilce_mean = ilce_data[\'bildirimsiz_sum\'].mean()\n    outliers = ilce_data[ilce_data[\'bildirimsiz_sum\'] > ilce_mean]\n    outliers_count = len(outliers)\n    outliers_info[ilce] = outliers_count\n\n    # Aykırı değerleri ilçenin ortalama değeri ile eşitleme\n    sub_train.loc[outliers.index, \'bildirimsiz_sum\'] = ilce_mean\n\n# Her ilçede kaç adet aykırı değer tespit edildiğini yazdırma\nprint("İlçe Aykırı Değer Bilgileri:")\nfor ilce, count in outliers_info.items():\n    print(f"{ilce}: {count} adet aykırı değer tespit edildi.")'

# Model değişti

In [58]:
#merged_df = pd.merge(train, sadelesmis_df, on=['tarih','ilce'], how='left')

sub_train_start = '2023-01-01'
sub_train_end = '2024-01-31'

sub_start_date = '2024-02-01'
sub_end_date = '2024-02-29'

# Tarih aralığına göre veri setini filtreleme
sub_train = merged_df[(merged_df['tarih'] >= sub_train_start) & (merged_df['tarih'] <= sub_train_end)]
sub_merged = merged_df[(merged_df['tarih'] >= sub_start_date) & (merged_df['tarih'] <= sub_end_date)]

In [59]:
from sklearn.linear_model import ElasticNet

# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma
train_data = sub_train

# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma
test_data = sub_merged

# Tahminde bulunacak ilçelerin listesi
ilceler = sub_train['ilce'].unique()

# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım
predictions = {}

# StandardScaler nesnesi oluşturma ve eğitme
scaler = StandardScaler()
scaler.fit(train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))

# Her bir ilçe için tahminde bulunma
for ilce in ilceler:
    ilce_train_data = train_data[train_data['ilce'] == ilce]
    ilce_test_data = test_data[test_data['ilce'] == ilce]
    
    # Standartlaştırma işlemi
    features_train_standardized = scaler.transform(ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    features_test_standardized = scaler.transform(ilce_test_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    
    # Hedef değişken
    target = ilce_train_data['bildirimsiz_sum']
    
    # Modeli eğitme (ElasticNet regresyonu)
    model = ElasticNet(alpha=0.1, l1_ratio=0.5)  # İstediğiniz alpha ve l1_ratio değerlerini belirtin
    model.fit(features_train_standardized, target)
    
    # Tahminde bulunma
    ilce_predictions = model.predict(features_test_standardized)
    
    # Tahminleri depolama
    predictions[ilce] = ilce_predictions

In [60]:
"""from sklearn.linear_model import ElasticNet

# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma
train_data = sub_train

# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma
test_data = sub_merged

# Tahminde bulunacak ilçelerin listesi
ilceler = sub_train['ilce'].unique()

# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım
predictions = {}

# Her bir ilçe için tahminde bulunma
for ilce in ilceler:
    ilce_train_data = train_data[train_data['ilce'] == ilce]
    ilce_test_data = test_data[test_data['ilce'] == ilce]
    
    # Tahminde kullanılacak özellikler (tüm sütunlar)
    features = ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])
    
    # Hedef değişken
    target = ilce_train_data['bildirimsiz_sum']
    
    # Modeli eğitme (ElasticNet regresyonu)
    model = ElasticNet(alpha=0.1, l1_ratio=0.1)  # İstediğiniz alpha ve l1_ratio değerlerini belirtin
    model.fit(features, target)
    
    # Tahminde bulunma
    ilce_predictions = model.predict(ilce_test_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum']))
    
    # Tahminleri depolama
    predictions[ilce] = ilce_predictions"""

"from sklearn.linear_model import ElasticNet\n\n# Tahmin edilecek tarih aralığındaki verileri eğitim verisi olarak kullanma\ntrain_data = sub_train\n\n# Tahmin edilecek tarih aralığındaki verileri test verisi olarak kullanma\ntest_data = sub_merged\n\n# Tahminde bulunacak ilçelerin listesi\nilceler = sub_train['ilce'].unique()\n\n# Tahminlerin ve gerçek değerlerin depolanacağı bir sözlük oluşturalım\npredictions = {}\n\n# Her bir ilçe için tahminde bulunma\nfor ilce in ilceler:\n    ilce_train_data = train_data[train_data['ilce'] == ilce]\n    ilce_test_data = test_data[test_data['ilce'] == ilce]\n    \n    # Tahminde kullanılacak özellikler (tüm sütunlar)\n    features = ilce_train_data.drop(columns=['tarih', 'ilce', 'bildirimsiz_sum'])\n    \n    # Hedef değişken\n    target = ilce_train_data['bildirimsiz_sum']\n    \n    # Modeli eğitme (ElasticNet regresyonu)\n    model = ElasticNet(alpha=0.1, l1_ratio=0.1)  # İstediğiniz alpha ve l1_ratio değerlerini belirtin\n    model.fit(featur

In [61]:
from sklearn.metrics import mean_absolute_error

# MAE skorlarını depolamak için bir sözlük oluşturalım
mae_scores = {}

# Her bir ilçe için MAE skorunu hesaplayalım
for ilce, ilce_predictions in predictions.items():
    true_values = test_data[test_data['ilce'] == ilce]['bildirimsiz_sum'].values
    mae = mean_absolute_error(true_values, ilce_predictions)
    mae_scores[ilce] = mae

# Her ilçenin MAE skorunu yazdıralım
print("İlçe Bazlı MAE Skorları:")
for ilce, mae in mae_scores.items():
    print(f"{ilce}: {mae}")

İlçe Bazlı MAE Skorları:
izmir-aliaga: 0.40248269300561723
izmir-bayindir: 0.257039813269561
izmir-bayrakli: 0.3741439481939862
izmir-bergama: 0.34652753170741724
izmir-bornova: 0.33624245591402196
izmir-buca: 0.1673861805621645
izmir-cesme: 0.2899412553974249
izmir-cigli: 0.6159741057668685
izmir-dikili: 0.41486987209752296
izmir-foca: 0.7778630553696777
izmir-gaziemir: 0.7299685272271708
izmir-guzelbahce: 0.5901465574384843
izmir-karabaglar: 0.30326641399426996
izmir-karaburun: 0.31211236805666925
izmir-karsiyaka: 0.28116413281640573
izmir-kemalpasa: 0.33152358782805436
izmir-kinik: 0.43601402861055716
izmir-kiraz: 0.05409926953689534
izmir-konak: 0.13210119609384066
izmir-menderes: 0.18848073212396882
izmir-menemen: 0.5582220421491434
izmir-odemis: 0.14371106597361927
izmir-seferihisar: 0.5086166942459194
izmir-tire: 0.04198873569830792
izmir-torbali: 0.17739344998378334
izmir-urla: 0.30719098795998484
manisa-akhisar: 0.12333267447600912
manisa-alasehir: 0.35865412550316045
manisa-d

In [62]:
# Tüm MAE skorlarının ortalamasını hesaplayalım
overall_mae = sum(mae_scores.values()) / len(mae_scores)

# Ortalama MAE skorunu yazdıralım
print("Tüm İlçelerin Ortalama MAE Skoru:", overall_mae)

# ESKİDEN 5.30 ALIYORDUM? SORUN NEDİR

Tüm İlçelerin Ortalama MAE Skoru: 0.3247298606719841


In [63]:
print("0.01-0.01 ikilisi : 0.4155740614702738")

0.01-0.01 ikilisi : 0.4155740614702738


#### rmse

In [64]:
from sklearn.metrics import mean_squared_error

ilce_rmse = {}
for ilce, ilce_test_data in test_data.groupby('ilce'):
    true_values = ilce_test_data['bildirimsiz_sum']
    
    predicted_values = predictions[ilce]
    
    rmse = np.sqrt(mean_squared_error(true_values, predicted_values))
    
    ilce_rmse[ilce] = rmse

for ilce, rmse in ilce_rmse.items():
    print(f"{ilce}: RMSE = {rmse}")

izmir-aliaga: RMSE = 0.4086831540899204
izmir-balcova: RMSE = 0.3464340912770593
izmir-bayindir: RMSE = 0.26059804869550945
izmir-bayrakli: RMSE = 0.3784509589713263
izmir-bergama: RMSE = 0.350464210957406
izmir-beydag: RMSE = 0.05926632362683479
izmir-bornova: RMSE = 0.3382132759880828
izmir-buca: RMSE = 0.16926707099189203
izmir-cesme: RMSE = 0.2909011802512968
izmir-cigli: RMSE = 0.6195662627755351
izmir-dikili: RMSE = 0.4192279929642327
izmir-foca: RMSE = 0.7784988361051819
izmir-gaziemir: RMSE = 0.7364778295831194
izmir-guzelbahce: RMSE = 0.604132342998196
izmir-karabaglar: RMSE = 0.3062713099200394
izmir-karaburun: RMSE = 0.34857882867568607
izmir-karsiyaka: RMSE = 0.2834028558656881
izmir-kemalpasa: RMSE = 0.33312608598800103
izmir-kinik: RMSE = 0.44257603794231465
izmir-kiraz: RMSE = 0.10211647701726717
izmir-konak: RMSE = 0.13384820202287762
izmir-menderes: RMSE = 0.19400443045761237
izmir-menemen: RMSE = 0.5619965289044996
izmir-narlidere: RMSE = 0.660887084292415
izmir-odemi

In [65]:
# Tüm ilçelerin RMSE değerlerinin ortalamasını hesapla
mean_rmse = np.mean(list(ilce_rmse.values()))

# Ortalama RMSE değerini yazdır
print("Tüm ilçelerin Ortalama RMSE değeri:", mean_rmse)

Tüm ilçelerin Ortalama RMSE değeri: 0.3299277086194426


In [66]:
print("0.01-0.01 ikilisi : 0.43664179855328844")

0.01-0.01 ikilisi : 0.43664179855328844


# Model katsayı (değişim ilk değere aşırı overfit oldu)

In [67]:
"""# Model katsayılarını al
coefficients = model.coef_

# Özellik isimlerini al
feature_names = features.columns

# Katsayıları ve özellik isimlerini eşleştir
coefficients_dict = dict(zip(feature_names, coefficients))

# Katsayıları DataFrame'e dönüştür
coefficients_df = pd.DataFrame(list(coefficients_dict.items()), columns=['Feature', 'Coefficient'])

# Katsayıları mutlak değerlerine göre sırala
coefficients_df['Absolute_Coefficient'] = np.abs(coefficients_df['Coefficient'])
coefficients_df = coefficients_df.sort_values(by='Absolute_Coefficient', ascending=False)

# Önem değerlerini ısı haritası olarak çiz
plt.figure(figsize=(10, 6))
sns.heatmap(coefficients_df.set_index('Feature')[['Absolute_Coefficient']], cmap='coolwarm', annot=True, fmt=".3f", linewidths=.5)
plt.title('Katsayıların Mutlak Değerleri (Özellik Önemleri)')
plt.xlabel('Önem')
plt.ylabel('Özellikler')
plt.show()"""

'# Model katsayılarını al\ncoefficients = model.coef_\n\n# Özellik isimlerini al\nfeature_names = features.columns\n\n# Katsayıları ve özellik isimlerini eşleştir\ncoefficients_dict = dict(zip(feature_names, coefficients))\n\n# Katsayıları DataFrame\'e dönüştür\ncoefficients_df = pd.DataFrame(list(coefficients_dict.items()), columns=[\'Feature\', \'Coefficient\'])\n\n# Katsayıları mutlak değerlerine göre sırala\ncoefficients_df[\'Absolute_Coefficient\'] = np.abs(coefficients_df[\'Coefficient\'])\ncoefficients_df = coefficients_df.sort_values(by=\'Absolute_Coefficient\', ascending=False)\n\n# Önem değerlerini ısı haritası olarak çiz\nplt.figure(figsize=(10, 6))\nsns.heatmap(coefficients_df.set_index(\'Feature\')[[\'Absolute_Coefficient\']], cmap=\'coolwarm\', annot=True, fmt=".3f", linewidths=.5)\nplt.title(\'Katsayıların Mutlak Değerleri (Özellik Önemleri)\')\nplt.xlabel(\'Önem\')\nplt.ylabel(\'Özellikler\')\nplt.show()'

In [68]:
# Tahminlerin depolanacağı DataFrame'i oluşturalım
tahminler_df = pd.DataFrame(columns=['Tarih', 'İlçe', 'Tahmin'])

# Her bir unique ilçe için tahmin skorlarını sırayla ekleyelim
for ilce, ilce_predictions in predictions.items():
    # Tahminlerin başlangıç tarihi
    start_date = pd.Timestamp('2024-02-01')
    
    # Tahmin skorlarını sırayla DataFrame'e ekleyelim
    for prediction in ilce_predictions:
        tahminler_df = pd.concat([tahminler_df, pd.DataFrame({'Tarih': [start_date], 'İlçe': [ilce], 'Tahmin': [prediction]})], ignore_index=True)
        start_date += pd.Timedelta(days=1)

/tmp/ipykernel_18/204946834.py:11: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tahminler_df = pd.concat([tahminler_df, pd.DataFrame({'Tarih': [start_date], 'İlçe': [ilce], 'Tahmin': [prediction]})], ignore_index=True)


In [69]:
tahminler_df['unique_id'] = tahminler_df['Tarih'].astype(str) + '-' + tahminler_df['İlçe']
tahminler_df = tahminler_df.rename(columns={'Tahmin': 'bildirimsiz_sum'})

In [70]:
tahminler_df = tahminler_df.drop(columns=['Tarih','İlçe'])
tahminler_df = tahminler_df[['unique_id', 'bildirimsiz_sum']]

In [71]:
tahminler_df

,unique_id,bildirimsiz_sum
0,2024-02-01-izmir-aliaga,-0.040922
1,2024-02-02-izmir-aliaga,0.414603
2,2024-02-03-izmir-aliaga,0.410521
3,2024-02-04-izmir-aliaga,0.411232
4,2024-02-05-izmir-aliaga,0.395442
...,...,...
1358,2024-02-25-manisa-ahmetli,0.106442
1359,2024-02-26-manisa-ahmetli,0.106442
1360,2024-02-27-manisa-ahmetli,0.106442
1361,2024-02-28-manisa-ahmetli,0.106442


In [72]:
tahminler_df.to_csv('submission.csv', index=False)